In [1]:
%env THEANO_FLAGS=device=gpu0,floatX=float32
from ADsnetwork import LeMoN_AI

env: THEANO_FLAGS=device=gpu0,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [2]:
model = LeMoN_AI(True, True)

Music conv:  (None, 256, 3)
Shift conv:  (None, 256, 10)
Position pool:  (None, 512)
Conc:  (None, 1024)
Output:  (None, 114)


In [3]:
from time import time
from prefetch_generator import BackgroundGenerator
from telebot import TeleBot
from utils import get_data, iterate_minibatches
from utils import *
import config
import numpy as np

bot = TeleBot(config.telegram_token)

In [4]:
def send_log(message):
    bot.send_message(config.channel, message)

In [5]:
send_log("Презапускаем")

In [6]:
epoch = 100
batch_size = 200
time_leght = 20

In [7]:
points, music = get_data("ok_dat.npy")

In [8]:
iterate_mini_set = {
    "points": points[:1],
    "music": music[:1]
}

In [9]:
message = """Epoch: {}
Time: {}
Mean loss(square error): {}"""

In [10]:
send_log("Начинаем шоу")

In [11]:
c_batch_per_epoch = 100

In [12]:
model.load()

In [13]:
def iterate_minibatches(points, music):
    pool = Pool(16)
    spp = "spectr/0.0.spec"
    points = points[0]
    music = music[0]
    sh = positions_to_moves(points)
    ind = np.arange(0, len(music)-20)
    np.random.shuffle(ind)
    for i in range(0, len(ind), 100):
        mus = np.array(pool.map(lambda x: np.load(spp.format(x)), ind[i:i+100]))
        shift, delta = pool.map(lambda x:sh[x:x+19], ind[i:i+100]), pool.map(lambda x:sh[x], ind[i:i+100])
        spos = pool.map(lambda x:points[x], ind[i:i+100])
        yield np.array(mus), np.array(shift), np.array(spos), np.array(delta)

In [ ]:
try:
    for epoch in range(1, epoch+1):
        n = 0
        loss = 0
        
        st = time()
        try:
            for i, (music_, shift, stp, delta_mov) in enumerate(iterate_minibatches(**iterate_mini_set)):
                if i+1 > c_batch_per_epoch:
                    break
                loss += model.train(music_.reshape((-1, 4, 200)), shift.reshape((-1, 19, 38*3)), stp, delta_mov.reshape((-1, 38*3)))
#                 loss += model.train(music_, shift, stp, delta_mov)
                del music_, shift, stp, delta_mov
                n += 1
        except IndexError as er:
            send_log("Index error in batch {}".format(er))
        t = time() - st
        send_log(message.format(epoch, t, loss/(n*1.)))
        send_log("Сохраняем веса")
        try:
            model.save()
        except Exception as e:
            send_log("Error when save: {}".format(e))
except Exception as e:
    send_log("Error: {}".format(e))

In [16]:
for i, (music_, shift, stp, delta_mov) in enumerate(iterate_minibatches(**iterate_mini_set)):
    break

KeyboardInterrupt: 

In [ ]:
model.save()

In [15]:
model.train(music_.reshape((-1, 4, 200)), shift.reshape((-1, 19, 38*3)), stp, delta_mov.reshape((-1, 38*3)))

NameError: name 'music_' is not defined

In [ ]:
model.predict(music_, shift, stp)

In [ ]:
music_.shape, shift.shape, stp.shape, delta_mov.shape

In [ ]:
music_.reshape((-1, 4, 200)).shape

In [15]:
model.train(music_.reshape((-1, 20, 200)), shift.reshape((-1, 19, 38*3)), stp, delta_mov.reshape((-1, 38*3)))

NameError: name 'music_' is not defined